# Extraction, Transform, and Load

ETL is a common necessity for data engineering and data processing pipelines.
The source of the data may be other structured databases, unstructured data stores, data APIs, etc.

ETL can be a simple data acquisition task, such as shown below.

![AutomatedDataAcquisition.png MISSING](../images/AutomatedDataAcquisition.png)

**Or, it may be part of larger process to accumulated data and information in support of advanced analytical systems.**

![AutomatedDataAcquisition_to_Analytics.png MISSING](../images/AutomatedDataAcquisition_to_Analytics.png)

---

## In the context of ETL, you now have the tools to perform this activity.

In the data loading lab, you read in two data files into the Pandas dataframes and prepare the data for loading. You then loaded the database with the data. To design and load a database, we need to understand how to acquire data from a remote resource, such as the web or an API and process it with Pandas.

In the loading lab we explored psycopg2 for loading database. In this notebook we will see how to use the SQLAlchemy library to simplify data loading. More specifically, we will extract data from a wikipage, transform the data into pandas dataframe, and then load the database directly from the pandas dataframe. This lab is an example of automating each step of ETL. 


## Tasks:

 **Consider**:
 + https://en.wikipedia.org/wiki/Land_use_statistics_by_country   
 
In the cells below, 

 1. Define a table for information about the worlds countries.
 1. Describe some challenges you foresee with the data
 1. Review and modify code cells that pull down the data from the tables into a data frame
 1. Load the data into your database
 1. Test loaded data with SQL queries

### 1. Define Tables

We will be using SQLAlchemy for creating a table and loading data into the table. For programmatic access you have two options: psycopg2 and sqlalchemy. The later is more powerful. If you plan to stick to one, then I think sqlalchemy is the right option. 

Now we will create a query string with the above create statement and execute the query with a sqlalchemy engine.

In [ ]:
import getpass
mypasswd = getpass.getpass()
username = 'SSO'
host = 'pgsql.dsa.lan'
database = 'dsa_student'

In [ ]:
# Then connects to the DB
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

# SQLAlchemy Connection Parameters
postgres_db = {'drivername': 'postgres',
               'username': username,
               'password': mypasswd,
               'host': host,
               'database' :database}
engine = create_engine(URL(**postgres_db), echo=True)
del mypasswd

In [ ]:
query = """
DROP TABLE IF EXISTS land_use_statistics;
CREATE TABLE land_use_statistics (
    country              varchar(100), -- Character String, varied length
    rank                 INT NOT NULL, -- Integer
    cultivated_land_k    INT NOT NULL,
    cultivated_land_p    REAL NOT NULL, -- Floating point number
    arable_land_k        INT NOT NULL,
    arable_land_p        REAL NOT NULL,
    permanent_crops_k    INT NOT NULL,
    permanent_crops_p    REAL NOT NULL,
    other_land_k         REAL NOT NULL,
    other_land_p         REAL NOT NULL,
    total_area           REAL NOT NULL,
    date_yr              INT NOT NULL,
    CONSTRAINT pk_land_use_statistics
     PRIMARY KEY (country)
)
"""

with engine.connect() as connection:
    res = connection.execute(query)
    print(res)

### 2. Describe the challenges you foresee with the wikipedia data

### 3. Data Scrapping 

In [ ]:
#import the library to query a website
import requests
# import Beautiful soup library to access functions to parse the data returned from the website
from bs4 import BeautifulSoup



In [ ]:
# specify the url
url = "https://en.wikipedia.org/wiki/Land_use_statistics_by_country"
# Open website URL and return the html to the variable 'response'
response = requests.get(url)
print(response.encoding)
print(response.status_code)

The response we get from web is typically html content. 
We can read the content of the server's response. 
Below, when a `BeautifulSoup` object is created from an html response, we explicitly reference the text format(`response.text`).

The default encoding format is 'UTF-8' as shown below. 

[Click here for additional documentations about the response object.](http://docs.python-requests.org/en/master/user/quickstart/#response-content)



In [ ]:
# Parse the html in the 'response' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(response.text, "html")

#####  Basic Inspection
Use `prettify` function to print the data in its nested html structured format.

In [ ]:
print(soup.prettify)

We need to extract the table which has land usage for each country. This table should be present in one of the html tags.

We can work with the tags to extract data present in them.  
"**soup.tag**": will return the content between opening and closing tag including tag. 

Additionally, the `.string` value is the data between the tags.
Compare the two cells below.

In [ ]:
print(soup.title)
print(soup.title.string)

**Identify the html tag**: 
The data is in a table. 
You can use inspect element option when you right click the mouse to identify the tag which has the data. 

 * [Additional guide on webpage inspection](../resources/AnalyzingHTMLwithTheWebInspector.pdf)


<img src="../images/Wikipedia_Inspect_Screen.png">

**If we look at the inspected HTML source for the table,** 
abbreviated here to focus on the top two rows of data.

```HTML
<table class="sortable wikitable jquery-tablesorter">

 <thead>
     <tr bgcolor="#ececec" valign="top">
         <th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Rank</th>
         <th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Country</th>
         <th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Cultivated <br> land <br> (km<sup>2</sup>)</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Cultivated <br> land <br> (%)</th>
         <th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Arable <br> land <br> (km<sup>2</sup>)</th>
         <th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Arable <br> land <br> (%)</th>
         <th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Permanent <br> crops <br> (km<sup>2</sup>)</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Permanent <br> crops <br> (%)</th>
         <th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Other <br> lands <br> (km<sup>2</sup>)</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Other <br> lands <br> (%)</th>
         <th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Total <br> area <br> (km<sup>2</sup>)</th>
         <th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Date
</th>
     </tr>
 </thead>
 <tbody>
<tr>
    <td>—</td>
    <td><span class="flagicon" style="padding-left:25px;">&nbsp;</span><b><a href="/wiki/World" title="World">World</a></b></td>
    <td>17,235,800</td>
    <td>11.6</td>
    <td>15,749,300</td>
    <td>10.6</td>
    <td>1,549,600</td>
    <td>1</td>
    <td>131.701.100</td>
    <td>88.4</td>
    <td>149,000,000</td>
    <td>2011
    </td>
</tr>
<tr>
    <td>1</td>
    <td><span class="flagicon"><img alt="" src="//upload.wikimedia.org/wikipedia/en/thumb/4/41/Flag_of_India.svg/23px-Flag_of_India.svg.png" decoding="async" width="23" height="15" class="thumbborder" srcset="//upload.wikimedia.org/wikipedia/en/thumb/4/41/Flag_of_India.svg/35px-Flag_of_India.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/4/41/Flag_of_India.svg/45px-Flag_of_India.svg.png 2x" data-file-width="1350" data-file-height="900">&nbsp;</span><a href="/wiki/India" title="India">India</a></td>
    <td>1,891,761</td>
    <td>57</td>
    <td>1,753,694</td>
    <td>52.8</td>
    <td>138,067</td>
    <td>4.2</td>
    <td>1,395,502</td>
    <td>43</td>
    <td>3,287,263</td>
    <td>2011
    </td>
</tr>
<tr>
    <td>2</td>
    <td><span class="flagicon"><img alt="" src="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" decoding="async" width="23" height="12" class="thumbborder" srcset="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" data-file-width="1235" data-file-height="650">&nbsp;</span><a href="/wiki/United_States" title="United States">United States</a></td>
    <td>1,681,826</td>
    <td>17.1</td>
    <td>1,652,028</td>
    <td>16.8</td>
    <td>29,798</td>
    <td>0.3</td>
    <td>8,151,691</td>
    <td>82.9</td>
    <td>9,833,517</td>
    <td>2011
    </td>
</tr>
<tr>
...
</tr></tbody><tfoot></tfoot></table>
```

We see that the table tag has class settings of:
 * sortable 
 * wikitable 
 * jquery-tablesorter
 
```HTML
<table class="sortable wikitable jquery-tablesorter">
```

We want to focus on the `wikitable`.  

In [ ]:
# We can fetch all Tables with a find_all() 
all_tables=soup.find_all('table')
# print(type(all_tables))
print(f"Num of avaiable tables = {len(all_tables)}")

# We can find all of the tables with wikitable class 
wikitables=soup.find_all('table', {'class':"wikitable"})
print(f"Num of avaiable wikitables = {len(wikitables)}")
right_table = wikitables[0]
# print(type(right_table))


The `Tag` element is the table.

**Examining the HTML Table Header, we have these columns**

 * Rank
 * Country
 * Cultivated Land km^2
 * Cultivated Land %
 * Arable Land km^2
 * Arable Land %
 * Permanent Crops km^2
 * Permanent Crops %
 * Other lands km^2
 * Other lands %
 * Total Area
 * Date

Therefore, a simple approach is to iterate through the HTML table rows, the `<tr>...</tr>` and process the data elements.

Reviewing the HTML above, we see we need to skip the headers and the "World" row.

Additionally, we will stop processing when we get out of the ranked rows, that is when Rank is not a number.


In [ ]:
# We will use the locale library so we can use 
# atof and atoi to convert alphanumeric to float and integers, respectively.
import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

rank=[]
country=[]
cultivated_land_k=[]
cultivated_land_p=[]
arable_land_k=[]
arable_land_p=[]
permanent_crops_k=[]
permanent_crops_p=[]
other_land_k=[]
other_land_p=[]
total_area=[]
date_yr=[]


# skip first iteration as we dont need headers
for row in right_table.findAll("tr")[1:]: 
    # for each row, pull out the td elements.
    cells = row.findAll('td') # To store all other details
    

#     print(f"cells = {cells}")    
    
    this_rank = cells[0].find(text=True)
    print("Processing rank {}".format(this_rank))

    # If the rank is a number, we can convert it
    if (not this_rank.isnumeric()):
        print("Non-Ranked, skipping")
        continue
        
    rank.append(locale.atoi(this_rank))

    # for the country name, we need to find the name (text) in the Country Hyperlink (a)
    countr_cell = cells[1].find('a').find(text=True)
    print(countr_cell)

    country.append(countr_cell)

    # Adjust the the data from Text to numeric data types
    cultivated_land_k.append(locale.atoi(cells[2].find(text=True)))
    cultivated_land_p.append(locale.atof(cells[3].find(text=True)))

    arable_land_k.append(locale.atoi(cells[4].find(text=True)))
    arable_land_p.append(locale.atof(cells[5].find(text=True)))

    permanent_crops_k.append(locale.atoi(cells[6].find(text=True)))
    permanent_crops_p.append(locale.atof(cells[7].find(text=True)))

    # Note, that this is to float because the vatican row has a non-int value
    other_land_k.append(locale.atof(cells[8].find(text=True)))
    other_land_p.append(locale.atof(cells[9].find(text=True)))

    total_area.append(locale.atof(cells[10].find(text=True)))
    date_yr.append(locale.atoi(cells[11].find(text=True)))


##### Now that we have built all our columns, stack into a data frame!

In [ ]:
import pandas as pd

df=pd.DataFrame({'country': country,
                'rank': rank,
                'cultivated_land_k': cultivated_land_k,
                'cultivated_land_p': cultivated_land_p,
                'arable_land_k': arable_land_k,
                'arable_land_p': arable_land_p,
                'permanent_crops_k': permanent_crops_k,
                'permanent_crops_p': permanent_crops_p,
                'other_land_k': other_land_k,
                'other_land_p': other_land_p,
                'total_area': total_area,
                'date_yr': date_yr
                })


In [ ]:
df.head()

In [ ]:
df.tail()

### Check our column data types!
Does this match the data types we sketched out in the `CREATE TABLE` statement above?
If you need to adjust the definition, this would be the time.
Alternatively, we can adjust the columns using Pandas techniques.

In [ ]:
df.dtypes

Once we have our Panda data frame and the SQL table inline, we can load it into the database.

---

### 4. Load the data into your database

This time, instead of the manual loading, we are going to use the SQLAlchemy library.


#### When you run the cell below, carefully examine the output so you see what the SQLAlchemy library is doing!

In [ ]:

## Now that SQLAlchemy is loaded, the to_sql function
df.to_sql('land_use_statistics', # The table to load
          engine,             # The engine created above
          schema= username,   # The schema where the table lives, our pawprint
          if_exists='append', # If the table is found, it would keep loading the end of table.
          index=False,        # Ignore creating an index for the index col in the dataframe
          chunksize=20)       # Do 20 records from the data frame at a time


**Note that if we had not defined the table `land_use_statistics` in Step 1, the above code would still create a table in the database and load with data from the dataframe.** In this case engine create a table with dataframe columns as attributes. But there is no easy way to define the primary key with this approach. We need to alter the table aftewards to include any primary key. 

### 5. Test loaded data with SQL queries


```SQL
select * from land_use_statistics limit 2;
```

---

If the data was not loaded, please restart from the top and carefully check and redo each step.


In [ ]:
with engine.connect() as connection:
    res = connection.execute("select * from land_use_statistics limit 2")
    for row in res:
        print(row)


#### Now that the data is loaded, let's pull it back out!

The following code shows how to load a data into a pandas dataframe without writing a raw query. 




In [ ]:
df_backout = pd.read_sql_table(
    'land_use_statistics',
    con = engine,             # The engine created above
    schema= username   # The schema where the table lives, our pawprint
)

In [ ]:
df_backout.head(10)

In [ ]:
df_backout.tail(10)

# Save your notebook, then `File > Close and Halt`

---